<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
import scipy as sp
# from scipy.sparse import csr_matrix, hstack
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

Reading original data

In [2]:
start = 2
end = 22
step = 2

In [3]:
PATH_TO_DATA = ('../../data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=range(start, end, step))
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=range(start, end, step))

Separate target feature 

In [4]:
y_train = train_df['target']

In [5]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [30]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35,2014-10-04 11:19:53,22,2014-10-04 11:19:54,321,2014-10-04 11:19:54,23,2014-10-04 11:19:54,2211,2014-10-04 11:19:54,6730,2014-10-04 11:19:54,21,2014-10-04 11:19:54,44582,2014-10-04 11:20:00,15336,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782,2014-07-03 11:00:53,782,2014-07-03 11:00:58,782,2014-07-03 11:01:06,782,2014-07-03 11:01:09,782,2014-07-03 11:01:10,782,2014-07-03 11:01:23,782,2014-07-03 11:01:29,782,2014-07-03 11:01:30,782,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55,2014-12-05 15:55:13,55,2014-12-05 15:55:14,55,2014-12-05 15:56:15,55,2014-12-05 15:56:16,55,2014-12-05 15:56:17,55,2014-12-05 15:56:18,55,2014-12-05 15:56:19,1445,2014-12-05 15:56:33,1445,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022,2014-11-04 10:03:19,50,2014-11-04 10:03:20,222,2014-11-04 10:03:21,202,2014-11-04 10:03:21,3374,2014-11-04 10:03:22,50,2014-11-04 10:03:22,48,2014-11-04 10:03:22,48,2014-11-04 10:03:23,3374,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301,2014-05-16 15:05:32,301,2014-05-16 15:05:33,66,2014-05-16 15:05:39,67,2014-05-16 15:05:40,69,2014-05-16 15:05:40,70,2014-05-16 15:05:40,68,2014-05-16 15:05:40,71,2014-05-16 15:05:40,167,2014-05-16 15:05:44


In [6]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
import pickle
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

(u'\u0432\u0441\u0435\u0433\u043e \u0441\u0430\u0439\u0442\u043e\u0432:', 48371)


,site
13852,i1-js-14-3-01-11544-644949368-i.init.cedexis-r...
6240,static.vol24.fr
39840,mars.nasa.gov
3899,scholar.google.fr
9677,9zouxfza1h.s.ad6media.fr


In [7]:
# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]
idx_split

253561

In [8]:
full_df.shape

(336358, 20)

In [9]:
full_sites = full_df[sites]

In [10]:
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178


In [11]:
full_sites_as_array = full_sites.values

In [12]:
full_sites_as_array.shape

(336358, 10)

In [13]:
sites_without_zero = [[elem for elem in row if elem] for row in full_sites_as_array]

In [47]:
'''
zz = []
for row in full_sites_as_array:
    new_row = []
    for x in row:
        if x:
            new_row.append(x)
    zz.append(new_row)
'''

'\nzz = []\nfor row in full_sites_as_array:\n    new_row = []\n    for x in row:\n        if x:\n            new_row.append(x)\n    zz.append(new_row)\n'

In [14]:
str_sessions = [' '.join([str(x) for x in row]) for row in sites_without_zero]

In [51]:
str_sessions[:5]

['718',
 '890 941 3847 941 942 3846 3847 3846 1516 1518',
 '14769 39 14768 14769 37 39 14768 14768 14768 14768',
 '782 782 782 782 782 782 782 782 782 782',
 '22 177 175 178 177 178 175 177 177 178']

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [15]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=100000)

In [16]:
X_tfidf = vectorizer.fit_transform(str_sessions)

Add features based on the session start time: hour, whether it's morning, day or night and so on.

In [18]:
np.__version__

'1.14.0'

In [19]:
new_feats = pd.DataFrame()

In [20]:
new_feats['year'] = full_df['time1'].apply(lambda t : t.year)
new_feats['month'] = full_df['time1'].apply(lambda t : t.month)

In [21]:
new_feats['year_month_start'] = full_df['time1'].apply(lambda ts: int(ts.year * 100 + ts.month))

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()
scaler.fit(new_feats['year_month_start'].values.reshape(-1, 1))
new_feats['year_month_start'] = scaler.transform(new_feats['year_month_start'].values.reshape(-1, 1))

In [24]:
new_feats['hour'] = full_df['time1'].apply(lambda tm : tm.hour)

In [25]:
MORNING = 5
DAY = 12
EVENING = 18

In [26]:
def get_daypart(data):
    X_night = [1 if x < MORNING else 0 for x in data]
    X_morning = [1 if MORNING <= x < DAY else 0 for x in data]
    X_day = [1 if DAY <= x < EVENING else 0 for x in data]
    X_evening = [1 if EVENING <= x else 0 for x in data]
    return X_night, X_morning, X_day, X_evening

In [27]:
X_night, X_morning, X_day, X_evening = get_daypart(new_feats['hour'])

In [28]:
new_feats['morning'] = X_morning
new_feats['day'] = X_day
new_feats['evening'] = X_evening
# new_feats['night'] = X_night

In [25]:
# new_feats['night'].value_counts()

In [82]:
# new_feats.drop('night', axis=1)

In [26]:
sorted_datetimes = sorted(full_df["time2"].values)

In [25]:
timestamp_diff = []
for row in full_df.values:
    initial_timestamp = row[1].dt.total_seconds()
    final_timestamp = row[1].dt.total_seconds()
    for i in range(3, 20, 2):
        timestamp = row[i].dt.total_seconds()
        if timestamp > 0:
            final_timestamp = timestamp
        else:
            break
    timediff.append(final_timestamp - initial_timestamp)

AttributeError: 'Timestamp' object has no attribute 'dt'

In [29]:
for i in range(2, 11):
    new_feats["diff%s" % i] = full_df["time%s" % i] - full_df["time1"]

In [30]:
new_feats["diff"] = np.nanmax(new_feats[["diff%s" % i for i in range(2, 11)]], axis=1)

In [150]:
new_feats[["diff%s" % i for i in range(2, 11)] + ["diff"]]

,diff2,diff3,diff4,diff5,diff6,diff7,diff8,diff9,diff10,diff
session_id,,,,,,,,,,
1,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,00:00:00,00:00:01,00:00:01,00:00:01,00:00:01,00:00:02,00:00:02,00:00:25,00:00:26,00:00:26
3,00:00:01,00:00:02,00:00:02,00:00:02,00:00:02,00:00:03,00:00:04,00:00:05,00:00:07,00:00:07
4,00:00:30,00:01:00,00:01:30,00:02:00,00:02:30,00:03:00,00:03:30,00:04:00,00:04:30,00:04:30
5,00:02:17,00:02:17,00:02:18,00:02:18,00:02:54,00:02:54,00:02:54,00:04:01,00:04:06,00:04:06
6,00:00:08,00:04:31,00:05:12,00:11:26,NaT,NaT,NaT,NaT,NaT,00:11:26
7,00:00:00,00:00:00,00:00:00,00:00:30,00:01:39,00:01:39,00:01:40,00:01:41,00:01:42,00:01:42
8,00:00:00,00:00:02,00:00:03,00:00:03,00:00:03,00:00:03,00:00:05,00:00:05,00:00:06,00:00:06
9,00:00:27,00:00:27,00:00:27,00:00:28,00:00:28,00:00:28,00:00:44,00:00:44,00:00:45,00:00:45


In [31]:
time_diff = np.array(map(lambda x: x if x >= 0 else 0, new_feats["diff"].values.astype("timedelta64[s]").astype("int64")))

In [32]:
time_diff.size

336358

In [29]:
time_diff[:10]

array([  0,  26,   7, 270, 246, 686, 102,   6,  45,  87], dtype=int64)

In [30]:
log_time_diff = np.log(time_diff + 1)

In [172]:
log_time_diff[:10]

array([0.        , 3.29583687, 2.07944154, 5.60211882, 5.50938834,
       6.53233429, 4.63472899, 1.94591015, 3.8286414 , 4.47733681])

In [31]:
new_feats['log_diff'] = log_time_diff

In [32]:
new_feats.head()

,year,month,scale_year,hour,morning,day,evening,diff2,diff3,diff4,diff5,diff6,diff7,diff8,diff9,diff10,diff,log_diff
session_id,,,,,,,,,,,,,,,,,,
1,2014,2,0.534195,10,1,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0.000000
2,2014,2,0.534195,11,1,0,0,00:00:00,00:00:01,00:00:01,00:00:01,00:00:01,00:00:02,00:00:02,00:00:25,00:00:26,00:00:26,3.295837
3,2013,12,-1.871975,16,0,1,0,00:00:01,00:00:02,00:00:02,00:00:02,00:00:02,00:00:03,00:00:04,00:00:05,00:00:07,00:00:07,2.079442
4,2014,3,0.534195,10,1,0,0,00:00:30,00:01:00,00:01:30,00:02:00,00:02:30,00:03:00,00:03:30,00:04:00,00:04:30,00:04:30,5.602119
5,2014,2,0.534195,10,1,0,0,00:02:17,00:02:17,00:02:18,00:02:18,00:02:54,00:02:54,00:02:54,00:04:01,00:04:06,00:04:06,5.509388


In [49]:
new_feats.head()

,year,month,scale_year,hour,morning,day,evening,diff2,diff3,diff4,diff5,diff6,diff7,diff8,diff9,diff10,diff,log_diff,year_month_start
session_id,,,,,,,,,,,,,,,,,,,
1,2014,2,0.534195,10,1,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0.000000,0.476232
2,2014,2,0.534195,11,1,0,0,00:00:00,00:00:01,00:00:01,00:00:01,00:00:01,00:00:02,00:00:02,00:00:25,00:00:26,00:00:26,3.295837,0.476232
3,2013,12,-1.871975,16,0,1,0,00:00:01,00:00:02,00:00:02,00:00:02,00:00:02,00:00:03,00:00:04,00:00:05,00:00:07,00:00:07,2.079442,-1.800775
4,2014,3,0.534195,10,1,0,0,00:00:30,00:01:00,00:01:30,00:02:00,00:02:30,00:03:00,00:03:30,00:04:00,00:04:30,00:04:30,5.602119,0.501532
5,2014,2,0.534195,10,1,0,0,00:02:17,00:02:17,00:02:18,00:02:18,00:02:54,00:02:54,00:02:54,00:04:01,00:04:06,00:04:06,5.509388,0.476232


Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [43]:
X_tfidf.shape, new_feats[['scale_year', 'morning', 'day', 'evening', 'log_diff']].values.shape

((336358, 100000), (336358, 5))

In [42]:
res_df.shape

(336358, 100005)

In [50]:
res_df = sp.sparse.hstack([X_tfidf,
#                    new_feats[['scale_year', 'morning', 'day', 'evening', 'log_diff']]])
                    new_feats[['morning', 'day', 'evening', 'log_diff', 'year_month_start']]])

Perform cross-validation with logistic regression.

In [34]:
skf = StratifiedKFold(n_splits=5)

In [35]:
gs = GridSearchCV(estimator=LogisticRegression(), param_grid={'C' : np.logspace(-3, 3, 10)}, cv=skf, scoring='roc_auc', n_jobs=-1)

In [48]:
idx_split

253561

In [54]:
gs.fit(res_df.tocsr()[:idx_split], y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([1.00000e-03, 4.64159e-03, 2.15443e-02, 1.00000e-01, 4.64159e-01,
       2.15443e+00, 1.00000e+01, 4.64159e+01, 2.15443e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [55]:
gs.grid_scores_

C:\Python27\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.69039, std: 0.00840, params: {'C': 0.001},
 mean: 0.80646, std: 0.00666, params: {'C': 0.004641588833612777},
 mean: 0.87977, std: 0.00606, params: {'C': 0.021544346900318832},
 mean: 0.93766, std: 0.00393, params: {'C': 0.1},
 mean: 0.96166, std: 0.00319, params: {'C': 0.46415888336127775},
 mean: 0.97458, std: 0.00157, params: {'C': 2.154434690031882},
 mean: 0.97771, std: 0.00154, params: {'C': 10.0},
 mean: 0.97497, std: 0.00154, params: {'C': 46.41588833612773},
 mean: 0.96953, std: 0.00164, params: {'C': 215.44346900318823},
 mean: 0.96062, std: 0.00213, params: {'C': 1000.0}]

Make prediction for the test set and form a submission file.

In [51]:
clf = LogisticRegression(C=10)
clf.fit(res_df.tocsr()[:idx_split], y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
pred = clf.predict_proba(res_df.tocsr()[idx_split:])[:, 1]

In [38]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [53]:
write_to_submission_file(pred, "assignment6_alice_submission_C10_year_month.csv")